# Group 07 - Code - Rotten Tomatoes Web Scraping
## Vedenikova_Vitalia_StudentB

In [ ]:
#Importing the necessary packages
import pandas as pd
import numpy as np
import pickle
import sys
import re
import bs4
import requests
from bs4 import BeautifulSoup
import time
import json
import os
import csv
import warnings
from scipy import stats
import matplotlib.pyplot as plt
from requests_html import HTMLSession
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
from ast import literal_eval
pd.set_option('display.max_colwidth', None) #allows to see all the columns when working with data frames.

## 1. Introduction

The **objective** of this scraping process is to scrape the **critics' reviews for films that won the Academy Awards or were nominated from 2009 until 2024** as well as their **characteristics** (director, producer, runtime, cast and crew, etc.) from the website Rotten Tomatoes (URL of the main page : https://www.rottentomatoes.com/) 

We will ultimately analyse this data to try to better understand what criteria are used to choose an Oscar winner from amongst the nominees.

Rotten Tomatoes is a website with frequent updates. It allows for user interactions (anyone can leave a movie review) and its objective is to publish movie ratings which it calculates and constantly updates on the basis of the reviews.  

Student A scraped the website of the Academy Awards and produced early stage 1 data containing the names of the movies we will be examining along with the year of the Oscar Ceremony to which they were nominated. Student C also used an API to extract the release dates of the movies along other information and produced an early stage 1 dataset. We will be using some columns of both these data sets to generate the URLs of the movies' pages on Rotten Tomatoes. 

## 2. URLs to access

### 2.1 Generating the URLs

Our first objective is to create a list of URLs for each movie. We use the column with movie names in Student A's data frame and the column with the release dates from Student C's data frame.

In [ ]:
#Import Student A's data frame
dfA= pd.read_csv("../Code/Data/StudentA_Early_Stage_1.csv")
dfA = dfA.rename(columns={'Movie Title':'Movie_Title'})

In [ ]:
#Import Student C's data frame and merge it with Student A's 
dfC= pd.read_csv("../Code/Data/StudentC_Early_Stage_1.csv")
dfC = dfC.rename(columns={'Title':'Movie_Title', 'Year':'Release_Year'})
df = pd.merge(dfA, dfC, on='Movie_Title', how='right')

#Keep only the useful columns
df=df.drop(df.loc[:, 'Released':'BoxOffice'].columns, axis=1)
df=df.drop(['Unnamed: 0_x','Oscar Ceremony Year','Unnamed: 0_y'], axis=1)
df

We have to format the URLs while taking the following into consideration:

- Usually, the URLs are formatted as follows: https://www.rottentomatoes.com/m/movie_name. Some movies have their year of release at the end of their URL like so: https://www.rottentomatoes.com/m/movie_name_2015. If an error is encountered, the release year has to be added at the end of the URL.
- The movie's release year is almost always the year before the Oscars' Ceremony took place. We did discover, however, that this is often not the case, hence the need for Student C to extract release dates from an API.
- When there are certain characters such as a dot or apostrophe in the movie name (for example: "4.1 Miles"), it is removed in the URL.
- If a movie name has parenthesis, the contents of the parenthesis are omitted from the URL link (for example: "Birdman or (The Unexpected Virtue of Ignorance)").
- Lastly, there are a lot of exceptions to the above rules. Some URLs are formatted differently and have to be taken into consideration.

In [ ]:
#Define the URLs that we will be accessing

def generate_movie_links(df) -> None:
    """function to add columns to the dataframe, reformat the movie names and create an URL for each movie in the dataframe"""
    df.insert(2,"URL_Movie_Names", True)
    df.insert(3,"Rottentomatoes_URLs", True)
    df['URL_Movie_Names'] = df['Movie_Title'].str.lower() 
    df['URL_Movie_Names'] = df['URL_Movie_Names'].str.split('(').str[0] #remove any content in parenthesis coming after the title
    df['URL_Movie_Names'] = df['URL_Movie_Names'].str.replace(' ', '_')
    df['URL_Movie_Names'] = df['URL_Movie_Names'].str.rstrip('_') #remove any underscore that would be placed at the end of the string

    # These characters are generally replaced by different ones
    Char_replacements = [('&', 'and'),('é', 'e'),('á', 'a'),('à', 'a'),('í', 'i'),('ì', 'i'),('ǎ', 'a'),('ó', 'o'),('...', '_'),
                          ('_-_', '_')]

    for i,j in Char_replacements:
        df['URL_Movie_Names'] = df['URL_Movie_Names'].str.replace(i, j)

    #some URLs cannot be fixed systematically and must be fixed separately
    
    URL_replacements = [('13_hours', '13_hours_the_secret_soldiers_of_benghazi'),('birdman_or', 'birdman'),('spider-man', 'spider_man'),
                        ('spider-verse', 'spider_verse'),('maggie_simpson_in', 'the_simpsons'),('ennemis_interieurs', 'enemies_within'),
                        ('everything_will_be_okay', 'grill_gut_alles_gut'),('the_white_tiger', 'the_white_tiger_2020'),
                        ('half-blood', 'half_blood'),('the_wife', 'the_wife_2018'),('lifeboat', 'lifeboat_2019'),
                        ('kung_fu_panda_2', 'kung_fu_panda_the_kaboom_of_doom'),('les_mis_rables', 'les_miserables'),
                        ('star_wars_episode_viii_the_last_jedi', 'star_wars_the_last_jedi'),('star_wars_episode_ix_the_rise_of_skywalker', 'star_wars_the_rise_of_skywalker'),
                        ('music_by_prudence', 'music_by_prudence_2011'),('once_upon_a_time__in', 'once_upon_a_time_in'),
                        ('the_baader_meinhof_complex', 'baader_meinhof_complex'),('the_curious_case_of_benjamin_button', 'curious_case_of_benjamin_button'),
                        ('the_last_station', 'last_station'),('golda', 'golda_2022'),('the_milk_of_sorrow', 'milk_of_sorrow'),('the_young_victoria', 'young_victoria'),
                        ('undefeated', 'undefeated_2012'),('one_night_in_miami_', 'one_night_in_miami'),('ala_kachuu', 'ala_kachuu_take_and_run'),('ave_maria', '771428498'),
                        ('changeling', '1191742_changeling'),('glass_onion', 'glass_onion_a_knives_out_mystery'),('joanna', '771467811'),('hellboy_ii', 'hellboy_2'),
                        ('the_100_yearold', 'the_100_year_old'),('the_betrayal', 'nerakhoon'),('the_present', 'the_present_0000'),('the_reader', 'reader'),
                        ('the_secret_in_their_eyes', 'secret_in_their_eyes'),('tick_tick__boom', 'tick_tick_boom')]
                        
    for i,j in URL_replacements:
        df['URL_Movie_Names'] = df['URL_Movie_Names'].str.replace(i, j)


    # These characters generally disappear from Rotten Tomatoes links, so we remove them
    lsch = "!?.’:',+/-\"" #characters to remove in the URL
    for ch in lsch:
        df['URL_Movie_Names'] = df['URL_Movie_Names'].str.replace(ch, '')
    
    #some additional separate adjustments (urls that must have hyphens for example)
    Additional_replacements = [('chinas_unnatural_disaster_the_tears_of_sichuan_province', 'chinas-unnatural-disaster-the-tears-of-sichuan-province'),
                               ('encounters_at_the_end_of_the_world', '1194818-encounters_at_the_end_of_the_world'),('fantastic_mr_fox', '1197696-fantastic_mr_fox'),
                               ('how_they_donut_a_matter_of_loaf_and_death', 'wallace_and_gromit_a_matter_of_loaf_and_death_pands'),
                               ('lavatory_lovestory', 'ubornaya-istoriya-lyubovnaya-istoriya'),('pitääkö_mun_kaikki_hoitaa', 'pitaako_mun_kaikki_hoitaa'),
                               ('smile_pinki', 'smile-pinki'),('the_100yearold_man', 'the_100_year_old_man'),('the_conscience_of_nhem_en', 'the-conscience-of-nhem-en'),
                               ('the_duchess', '10009493-duchess'),('the_final_inch', 'the-final-inch'),('the_last_truck_closing_of_a_gm_plant', 'the-last-truck-the-closing-of-a-gm-plant'),
                               ('the_lovely_bones', '1189344-lovely_bones'),('the_wolfman', '1194949-wolfman'),('tron_legacy', '10011582-tron_legacy'),
                               ('winters_bone', '10012136-winters_bone'),('wreckit_ralph', 'wreck_it_ralph'),('xmen_days_of_future_past', 'x_men_days_of_future_past'),
                               ('yespeople', 'ja_folkid'),('dimanchesunday', 'dimanche'),('ninetyfive_senses', 'ninety_five_senses'),
                               ('the_most_dangerous_man_in_america_daniel_ellsberg_and_the_pentagon_papers', 'daniel_ellsberg_doc'),
                               ('precious_based_on_the_novel_push_by_sapphire', 'precious'),('shangchi', 'shang_chi'),
                               ('star_wars_the_force_awakens', 'star_wars_episode_vii_the_force_awakens'),('the_cove', '1208882-cove'),
                               ('wall·e', 'wall_e'),
                               ('borat_subsequent_moviefilm_delivery_of_prodigious_bribe_to_american_regime_for_make_benefit_once_glorious_nation_of_kazakhstan', 'borat_subsequent_moviefilm'),
                               ('alice_in_wonderland', '1221547-alice_in_wonderland'),('a_single_man', '1218217-single_man'),
                               ('burma_vj_reporting_from_a_closed_country', 'burma-vj'),('departures', '10010675-departures'),
                               ('kontiki', 'kon_tiki'),('the_house_of_small_cubes', 'la-maison-en-petits-cubes'),
                               ('the_visitor', '10008820-visitor'),('wanted', '1174279-wanted'),
                               ('watu_wote_all_of_us', 'watu_wote'),('happygolucky', '1195103-happy_go_lucky'),
                               ('salt', '1213717-salt')]
    
    for i,j in Additional_replacements:
        df['URL_Movie_Names'] = df['URL_Movie_Names'].str.replace(i, j)
    
    #create the URLs
    df['Rottentomatoes_URLs'] = [ f'https://www.rottentomatoes.com/m/{x}' for x in df['URL_Movie_Names']] #create the URLs
    
generate_movie_links(df)

Here below we see the sort of result we can expect from running the above code:

In [ ]:
df[df['Movie_Title'] == 'Changeling']

In [ ]:
df[df['Movie_Title'] == 'Zootopia']

### 2.2 Fixing the URLs

We need to check if the URLs we defined work. To do so we loop through the links we generated. If we cannot scrape basic information from the page or if the release year on the page does not match the release year in our dataframe, we append the release year in our dataframe at the end of the URL. If some URLs are still not working at the end of this process, we put them in a separate txt file to be examined. 

**IMPORTANT NOTE** : As of 18.04.24 the below will no longer work with the URLs we previously defined since Rotten Tomatoes updated the movies' pages. 
As a consequence, we will be using a handful of links gathered from https://web.archive.org/ to demonstrate how the code runs.

In [ ]:
df= pd.read_csv("../Code/Data/df_waybackmachine_links.csv") # <---- this is the csv we will be using to show the code works

In [ ]:
#Trying to fix the URLs by using a separate function

myheaders = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) \ '
              'AppleWebKit/537.36 (KHTML, like Gecko) \ ' 		        
              'Chrome/84.0.4147.89 Safari/537.36'}

headers = {'User-Agent': myheaders}


def fix_urls(df):
    """function that fixes defective URLs in the dataframe by adding the release year at the end of the link and separating the links
    still defective for further investigation. There are two situations in which we add the year: 1) the link itself is defective since
    we retrieve nothing for movie data or 2)the release year on the movie page does not match the official release year of the film"""
    
    for row_index, row in df.iterrows(): 
        link = row['Rottentomatoes_URLs']
        page_text = requests.get(link, headers=myheaders).text
        soup = BeautifulSoup(page_text,'lxml')
        movie_characteristics = soup.find('div', class_= "panel-body content_body") 
        movie_release_year= soup.find('p', class_= "info", attrs={'data-qa':'score-panel-subtitle'})
        
        try:
            if(movie_characteristics == None):
                new_link = link+'_'+ str(row['Release_Year']) #fixing the link by adding the release year at the end
                df.loc[row_index,'Rottentomatoes_URLs'] =  new_link #defining the new link in the data frame
                page_text = requests.get(new_link, headers=myheaders).text
                soup = BeautifulSoup(page_text,'lxml')
                movie_characteristics = soup.find('div', class_= "panel-body content_body")
                if (movie_characteristics == None): #if some are still not working, we put them in a separate csv file
                    with open('links_to_fix.csv', 'a', newline='') as file:
                        file.write(f"{df.loc[row_index,'Movie_Title']}, {new_link},\n")
                    continue
            elif(int(movie_release_year.text[0:4]) != int(row['Release_Year'])): #we must compare integers
                new_link = link+'_'+ str(row['Release_Year']) #fixing the link by adding the release year at the end
                df.loc[row_index,'Rottentomatoes_URLs'] =  new_link #defining the new link in the data frame
                page_text = requests.get(new_link, headers=myheaders).text
                soup = BeautifulSoup(page_text,'lxml')
                movie_characteristics = soup.find('div', class_= "panel-body content_body")
                if (movie_characteristics == None): #if some are still not working, we put them in a separate csv file
                    with open('links_to_fix.csv', 'a', newline='') as file:
                        file.write(f"{df.loc[row_index,'Movie_Title']}, {new_link},\n") 
                    continue
        except ValueError: 
            new_link = link+'_'+ str(row['Release_Year']) #fixing the link by adding the release year at the end
            df.loc[row_index,'Rottentomatoes_URLs'] =  new_link #defining the new link in the data frame
            page_text = requests.get(new_link, headers=myheaders).text
            soup = BeautifulSoup(page_text,'lxml')
            movie_characteristics = soup.find('div', class_= "panel-body content_body")
            if (movie_characteristics == None): #if some are still not working, we put them in a separate csv file
                with open('links_to_fix.csv', 'a', newline='') as file:
                    file.write(f"{df.loc[row_index,'Movie_Title']}, {new_link},\n") 
                continue

fix_urls(df)

After examining the csv file containing all the defective links and manually searching for them in rottentomatoes, some adjustments were made and additional exceptions were added in the movie links generator function.

Here below you can see the result.
I made **two links purposefully erronous** to demonstrate that the function works properly: 

1) **Barbie movie** : there is just an error message on the page. The function checks that it cannot scrape any movie information. It appends the release date at the end of the link. It checks the new link and sees that it did not work. It adds the movie to the 'links_to_fix.csv'

2) **Lion King movie**: the link works, but it shows the animated movie instead of the live-action remake from 2019. The function compares the release date in the dataframe to the one on the page, sees that they do not match and appends the release date at the end of the link.  It checks the new link and sees that it did not work (since we are on web.archive.org). It adds the movie to the 'links_to_fix.csv'

In [ ]:
df

### 2.3 Removing some movies 

In our original data frame containing all movie titles and dates, 10 movies were not found on the website and are removed from the dataframe so that the functions we will use later on run smoothly and the sources of errors are easily identified:

In [ ]:
#10 rows are removed from the dataframe
df_fixed = pd.read_csv("../Code/Data/df_with_most_links_fixed.csv")
df_fixed = df_fixed.drop('Unnamed: 0', axis=1)
df_links = pd.read_csv("../Code/Data/links_to_fix_original.csv", header=None)
df_links.columns =['Movie_Title', 'Rottentomatoes_URLs']
df = pd.merge(df_links, df_fixed, on=['Movie_Title'], how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)
df = df.drop('Rottentomatoes_URLs_x', axis=1)
df = df.rename(columns={'Rottentomatoes_URLs_y': 'Rottentomatoes_URLs'})
df.to_csv('../Code/Data/df_links_all_ok_new.csv', sep=',',index=True, encoding='utf-8')
df

## 3. Scraping Basic Information About The Movies 

Now that we have our list of URLs, we will access each one and scrape the following information that is available on the page:
- "Tomatometer" score (the critics' average score)
- Critics Consensus (the critics' general assessment)
- Audience Score (the audience's average score)
- Synopsis
- Full cast
- Rating
- Genre
- Original Language
- Director
- Producer
- Writer
- Release Date (Theaters)
- Release Date (Streaming)
- Box Office (Gross USA)
- Runtime
- Distributor
- Production Co
- Sound Mix
- Etc.

Most of this data will also be queried by Student C in case the scraped data needs to be completed or corrected. 

Basic characteristics are stored in a single class 'panel-body content_body' and therefore have to not only be extracted but parsed as well. 

For the cast and crew's names and their actual roles in the movie production, the process was more complicated. They were in separate places and needed to be placed in two separate lists before being brought together in tuples by using zip().

**IMPORTANT NOTE** : As of 18.04.24 the below will no longer work with the URLs we previously defined since Rotten Tomatoes updated the movies' pages. 
As a consequence, we will be using a handful of links gathered from https://web.archive.org/ to demonstrate how the code runs.

In [ ]:
#df= pd.read_csv("df_links_all_ok.csv")  <--- this is our data set with all the Oscar movies and their links

In [ ]:
df= pd.read_csv("../Code/Data/df_waybackmachine_links.csv") # <---- this is the csv we will be using to show the code works

In [ ]:
#Defining additional columns in the data frame
basic_characteristics = ['Rating','Genre', 'Original Language', 'Director', 'Producer','Cast and Crew',
                         'Writer', 'Release Date (Theaters)', 'Release Date (Streaming)', 'Rerelease Date (Theaters)', 'Box Office (Gross USA)',
                         'Runtime', 'Distributor','Production Co', 'Sound Mix', 'Aspect Ratio', 'Synopsis', 'Tomatometer', 'Tomatometer State', 
                         'Audience Score', 'Critics Concensus']
new_vals = [None for item in basic_characteristics]
new_col = dict(zip(basic_characteristics, new_vals))
df = df.assign(**new_col)
#df.set_index('Movie_Title', inplace=True)


In [ ]:
#user agent info to distinguish our scraper from a malicious bot
myheaders = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) \ '
              'AppleWebKit/537.36 (KHTML, like Gecko) \ ' 		        
              'Chrome/84.0.4147.89 Safari/537.36'}

headers = {'User-Agent': myheaders}

pd.set_option('mode.chained_assignment', None) #to remove some warnings when extracting the crew information 

#Scraping each movie page's information using Beautiful Soup and Requests
def scrape_page_information (df) -> pd.DataFrame:
    """function to scrape the information on the movie's page and place it in a dataframe"""
    
    not_crew_members_list =[]#this list is created for the cleaning section
    
    for row_index, row in df.iterrows():  # we index the rows in the dataframe so that we can easily insert the scraped data where needed
        link = row['Rottentomatoes_URLs']
        page_text = requests.get(link, headers=myheaders, timeout=10).text  ###########
        soup = BeautifulSoup(page_text,'lxml')
        
        #We look for the most basic information first (director, producer, runtime, etc.)
        try:
            movie_characteristics = soup.find('div', class_= "panel-body content_body") # in this class we retrieve the basic information
            movie_item_labels = list(movie_characteristics.find_all('b', attrs={"data-qa":"movie-info-item-label"}))
            movie_item_values = movie_characteristics.find_all('span', attrs={"data-qa":"movie-info-item-value"})
        except AttributeError:
            continue
        parsed_data = {}
        for idx,label in enumerate(movie_item_labels):
            label = label.text.rstrip(':')
            lsch = ['\n', '  '] #a lot of extra spaces and apostrophes to remove which prevent correct parsing
            parsed_data[label] = movie_item_values[idx].text
            for ch in lsch:
                parsed_data[label] = parsed_data[label].replace(ch, '')            
            df.loc[row_index, label] = parsed_data[label] # Now we insert the data into the data frame (this is where iterrows is useful)
        
        #We look for the audiencescore, tomatometerscore, tomatometerstate: 
        attrs = soup.find('score-board-deprecated').attrs #this is a dictionary from which we retrieve values                 
        for attr in ['audiencescore', 'tomatometerscore', 'tomatometerstate']:
            df.loc[row_index, 'Tomatometer'] = attrs['tomatometerscore']
            df.loc[row_index, 'Tomatometer State'] = attrs['tomatometerstate']
            df.loc[row_index, 'Audience Score'] = attrs['audiencescore']

        #We look for the critics consensus 
        consensus = soup.find('span', attrs={'data-qa':'critics-consensus'}) 
        try:
            consensus = consensus.text
            df.loc[row_index, 'Critics Concensus'] = consensus
        except AttributeError:
            df.loc[row_index, 'Critics Concensus'] = 'NaN'
            
        #We look for the synopsis of the movie   
        synopsis = soup.find('p', attrs={'data-qa':'movie-info-synopsis'})
        try:
            synopsis = synopsis.text
            df.loc[row_index, 'Synopsis'] = synopsis
        except AttributeError:
            df.loc[row_index, 'Synopsis'] = 'NaN'

        #We look for all the crew members and their roles/jobs in the movie production 
        crew = soup.css.select('a > p', class_='p--small', attrs={'data-qa':'cast-crew-item-link', 'id':'cast-and-crew', 'href':True})
        role = soup.find_all('p', class_='p--small')
        
        roles_list=[]
        for r in role:
            roles_list.append(r.get_text(strip=True))
               
        crew_list=[]
        for c in crew[1:]:
            crew_list.append(c.get_text(strip=True))

        not_crew_members = crew_list[len(roles_list):]#We append this to 'not_crew_members_list' created for the cleaning section 
        for n in not_crew_members:
            not_crew_members_list.append(n)
        
        crew_list = crew_list[:len(roles_list)] #we have names in crew_list that are not actual crew members, 
                                                #so we select only the ones that correspond to each role
       
        tuples = zip(crew_list, roles_list) #we make a tuple. A dictionary cannot be used because some people have several jobs.  
        full_crew = [f'{pair[0]} ({pair[1]})' for pair in tuples] #we iterate over the tuple to format it as we wish
        full_crew = ', '.join(map(str, full_crew))
        
        df['Cast and Crew'] = df['Cast and Crew'].astype('object')
        df.at[row_index, 'Cast and Crew'] =full_crew # to populate the rows we cannot use "loc" because loc can refer to values 
                                                              # as well as rows and columns. A ValueError kept appearing as a consequence.   
                                                              # 'at' always refers to a single value, which is what we need for a list. 
    return df, not_crew_members_list


df, not_crew_members_list = scrape_page_information(df)


In [ ]:
df

In [ ]:
#To avoid running the function again, we pickle the set of our not_crew_members_list:

not_crew_members_list_set = list(set(not_crew_members_list))

with open('not_crew_pickle_extra.pkl', 'wb') as f:
    pickle.dump(not_crew_members_list_set,f)
f.close()

In [ ]:
#df= pd.read_csv("movie_page_data.csv")

## 4. Scraping Critics Reviews of the Movies

Now we need to obtain the critics' comments about each movie.

To do so, we first define the links we will be accessing. In this case, we simply need to add '/reviews?intcmp=rt-scorecard_tomatometer-reviews' at the end of the URLs we previously defined:

In [ ]:
df_links = pd.read_csv("../Code/Data/df_links_all_ok.csv")
df_links = df_links.drop('Unnamed: 0', axis=1)
df_links.insert(1, 'Reviews_URLs', True)
for row_index, row in df_links.iterrows():
        link = row['Rottentomatoes_URLs']
        df_links.loc[row_index, 'Reviews_URLs'] = link+'/reviews?intcmp=rt-scorecard_tomatometer-reviews'
links = df_links['Reviews_URLs']
df_links = pd.DataFrame(links)
df_links

For scraping the actual critics' reviews, we must load all of them. This means that we must first press the "load more" button until there is no longer a button to press. 
Then, we should scrape the needed data : 

- Name of the critic
- Name of the journal for which he writes
- Review content
- Whether the review is negative or positive ('rotten' or 'fresh')
- The date the review was posted

In [ ]:
#Scraping all of the critics' comments using Selenium 

#we define our headers for the selenium driver
chrome_options = webdriver.ChromeOptions()
myheaders = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
chrome_options.add_argument(f"user-agent={myheaders}")
#chrome_options.add_argument('--headless=new')  
headers = {'User-Agent': myheaders}

driver = webdriver.Chrome(options=chrome_options)

def scrape_critics_reviews (df_links, driver) -> pd.DataFrame:
    """function to scrape all the critics reviews by using Selenium and placing them in data frames which we then concatenate into a single
    dataframe"""
    
    data_frames_list=[] #this is where we store the data frames
    
    for row_index, row in df_links.iterrows():
        link = row['Reviews_URLs']
        driver.get(link)
        
        #keep clicking the 'load more' button as long as it exists to get all the comments 
        while True:
            time.sleep(1.5)
            try:
                driver.find_element(By.XPATH, '//*[@id="reviews"]/div[2]/rt-button').click()
            except (NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException):   #when the button disappears, we need to 'break' the loop
                break 

        #scrape each comment's author, the journal they write for, the content of the comment and also if they were 'fresh'(positive) 
        #or 'rotten' (negative)
        reviews = driver.find_elements(By.CLASS_NAME,'review-row')
        reviews_list = []
    
        for review in reviews:
            try:
                name_critic = review.find_element(By.CSS_SELECTOR,'a.display-name').text #the x-path showed almost no IDs and instead a lot of tags so it is easier to select elements by CSS_SELECTOR
                name_journal = review.find_element(By.CSS_SELECTOR,'a.publication').text
                comment = review.find_element(By.CSS_SELECTOR,'p.review-text').text
                positive_or_negative = review.find_element(By.CSS_SELECTOR,'score-icon-critic-deprecated[alignment=left]').get_attribute('state')
                date = review.find_element(By.CSS_SELECTOR,'span[data-qa=review-date]').text
            except (NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException):
                continue
            
            reviews_list.append({
                'Date':date, 'Author':name_critic, 'Journal':name_journal, 'Positive_Negative':positive_or_negative, 'Reviews':comment})
            
            df_temp = pd.DataFrame(reviews_list)
            df_temp.insert(1, 'Reviews_URLs', row['Reviews_URLs'])
            
         
        data_frames_list.append(df_temp)

    driver.quit() #once done we close the driver
    df_result = pd.concat(data_frames_list, ignore_index=True) #once we are done, we concatenate all the data frames together
    return df_result

df_result = scrape_critics_reviews(df_links, driver)
df_result

In [ ]:
#df_result.to_csv('data_all_reviews.csv', sep=',',index=True, encoding='utf-8')

## 5. Merge the Datasets - Movie Characterisitcs and Critics' Reviews

Finally, we merge the datasets for the movie page information and the dataset for the reviews. The result is the stage 1 dataset.

In [ ]:
df_result = pd.read_csv("../Code/Data/data_all_reviews.csv")
df_moviedata = pd.read_csv("../Code/Data/movie_page_data.csv")
df_moviedata = df_moviedata.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)

In [ ]:
reviews_characteristics = ['Reviews_URLs']
reviews_vals = [None for item in reviews_characteristics]
new_col = dict(zip(reviews_characteristics, reviews_vals))
df_moviedata = df_moviedata.assign(**new_col)
for row_index, row in df_moviedata.iterrows():
        link = row['Rottentomatoes_URLs']
        df_moviedata.loc[row_index, 'Reviews_URLs'] = link+'/reviews?intcmp=rt-scorecard_tomatometer-reviews'

In [ ]:
#Since we want to keep the movies that also do not have any reviews, we merge with the "left" method (on the movie characteristics data set) 

df_merged = pd.merge(df_moviedata, df_result, on='Reviews_URLs', how='left')
df_merged.to_csv('Vedenikova_Vitalia_studentB_stage1.csv', sep=',',index=True, encoding='utf-8')

## 6. Clean the Resulting Dataset

### 6.1 Encode the CSV file correctly
Some accented letters or other language specific symbols had not been properly encoded into the csv file (ex: "Régional" was "RÃ©gional").
We simply encode the file as 'utf-8-sig'

In [ ]:
df = pd.read_csv('Vedenikova_Vitalia_studentB_stage1.csv', low_memory=False)

In [ ]:
df.to_csv('Vedenikova_Vitalia_studentB_stage3.csv', sep=',',index=False, encoding='utf-8-sig')
df = pd.read_csv('Vedenikova_Vitalia_studentB_stage3.csv', low_memory=False)

### 6.2 Remove Columns containing unnecessary data and give standardized names to columns
We should remove:
- columns we created to generate links
- columns containing data we scraped just because it was contained in the same class as actually useful information.

We also standardize the columns' names.

In [ ]:
df = df.drop(['Unnamed: 0','Unnamed: 0.1','URL_Movie_Names', 'Reviews_URLs', 'Release_Year',
              'Rerelease Date (Theaters)', 'Release Date (Streaming)', 'Release Date (DVD)','Sound Mix', 'Aspect Ratio', 
              'View the collection'], axis=1)

df = df.rename(columns={'Date':'Review_Date', 'Original Language':'Original_Language', 'Cast and Crew': 'Cast_And_Crew', 
                        'Release Date (Theaters)': 'Release_Date_(Theaters)', 'Box Office (Gross USA)':'Box_Office_(Gross_USA)', 
                        'Production Co':'Production_Co','Tomatometer State':'Tomatometer_State', 
                        'Audience Score':'Audience_Score','Critics Concensus':'Critics_Concensus'})

In [ ]:
#result:
df.columns

### 6.3 Remove the extra spaces (such as \n, \s)

In [ ]:
#example of extra spaces we want to remove:
df['Cast_And_Crew'][2594]

In [ ]:
#Remove leading and trailing spaces
col_list = ['Synopsis','Cast_And_Crew']
df[col_list] = df[col_list].apply(lambda x : x.str.strip())

#Remove two or more spaces 
df = df.replace(r'\n{1,}\s{2,}', ' ', regex=True)

In [ ]:
#result:
df['Cast_And_Crew'][2594]

### 6.4 Remove Unnecessary Characters and Inaccurate Information for the Crew
- We remove the "limited" in dates of the the Release_Date_(Theater) column.
- In the Cast_And_Crew column, we had rare cases of not scraping persons' names, but names of objects of the same HTML class (ex: Awards Leaderboard: Top Movies of 2022 (Original Music)). These should be removed.

In [ ]:
#We remove the trailing characters after the release dates:
df['Release_Date_(Theaters)'] = df['Release_Date_(Theaters)'].str[0:12]

In [ ]:
#To remove innacurate information in the Cast_And_Crew column, we use the "not_crew_members_list" which we defined 
#while running our function "scrape_page_information" :

#We deserialize our variable:
with open('../Code/Data/not_crew_pickle_extra.pkl', 'rb') as f:
    not_crew_list = pickle.load(f) 
    

In [ ]:
#example of extra elements with'Awards Leaderboard: Top Movies of 2022 (Original Music)' at the end of the string:
df['Cast_And_Crew'][6915]

In [ ]:
#We remove innacurate cast information: 
for row_index, row in df.iterrows():
    for element in not_crew_list:
        try: 
            if element in row['Cast_And_Crew']:
                text= row['Cast_And_Crew']
                t = re.sub(rf'{element}.*\)','',text)[0:-2] #removing the elements and slicing to remove the comma at the end of the list 
                df.loc[row_index, 'Cast_And_Crew'] = t
        except TypeError:
            continue

In [ ]:
#result
df['Cast_And_Crew'][6915]

### 6.5 Handle Missing Values

- In the Cast_And_Crew column, there might be some empty brackets when the role of the crew member is not indicated in Rotten Tomatoes. These empty brackets should contain "NaN".
- A lot of information is missing for certain movies (ex: no rottentomatoes score, no box office revenue, no distributor, etc.). In these cases, we should make sure that the empty boxes contain "NaN" (even in the csv file) so as to be clearly identified. 
- If one of the rows is missing a review, it should be removed unless the movie it belongs to has in fact no reviews. In this case, it should be kept so that the movie's information is not lost and the movie is further discussed in the group work.


In [ ]:
# Placing NaN between empty brackets in the crew and cast lists :
df['Cast_And_Crew'] = df['Cast_And_Crew'].str.replace('()', '(NaN)')

# Make sure that all blank fields have NaN: 
df = df.fillna("NaN")


In [ ]:
# Remove rows with 'NaN' under "Review" when there is more than one review per movie:

# First, count the amount of reviews for each movie and identify all the movies with more than one review:
review_counts = df["Movie_Title"].value_counts().sort_index()
review_counts = review_counts.to_frame()

subcategory = review_counts[review_counts['count']>1]
subcategory = subcategory.reset_index()
subcategory_list = [movie for movie in subcategory['Movie_Title']]

# Then, remove the row where there is no review and the movie is part of our subcategory_list:
for row_index, row in df.iterrows():
    if df.at[row_index,'Reviews'] == 'NaN' and row['Movie_Title'] in subcategory_list:
        df.drop(row_index,inplace=True)
        

### 6.6 Standardize Data 
- Make sure the release dates and review dates are all in the same date format
- The runtime of the movies should be in minutes (a single time unit)
- The Box Office Revenues should be expressed as integers since we will be plotting this data

In [ ]:
def standardize_data (df) -> pd.DataFrame:
    '''Standardizes the data in the columns 'Release_Date_(Theaters)','Review_Date', 'Runtime' and 'Box_Office_(Gross_USA)' '''
    
    # Put the release dates and review dates in the same date format
    df[['Release_Date_(Theaters)','Review_Date']] = df[['Release_Date_(Theaters)','Review_Date']].apply(lambda x: pd.to_datetime(x, errors='coerce'))

    #Convert the runtime of the movies to minutes
    time = df['Runtime'].str.extract('(\d+)h', expand=False).astype(float) * 60 +  df['Runtime'].str.extract('(\d+)m', expand=False).astype(float)
    df['Runtime'] = time

    #Convert the box office revenues into integers
    df['Box_Office_(Gross_USA)'] = df['Box_Office_(Gross_USA)'].str.replace('$', '')
    df['Box_Office_(Gross_USA)'] = df['Box_Office_(Gross_USA)'].replace({'K': 'e+03', 'M': 'e+06'}, regex=True).astype(float) 
    
    return df

df = standardize_data(df)

### 6.7 Remove Duplicates
Some reviews appear twice and should be removed. The ones with 'NaN' should be kept.

In [ ]:
df = df.fillna("NaN")
df = pd.concat([df[df['Reviews']=='NaN'], df[df['Reviews']!='NaN'].drop_duplicates(['Reviews'])])

In [ ]:
#We check that we removed all duplicates except the 'NaN':
counts = df["Reviews"].value_counts().sort_index()
counts = counts.to_frame()
reviews_with_duplicates = counts[counts['count']>1]
reviews_with_duplicates

In [ ]:
df.sort_values(['Movie_Title','Review_Date'], inplace=True)

In [ ]:
df.to_csv('Vedenikova_Vitalia_studentB_stage3.csv', sep=',',index=False, encoding='utf-8-sig')

## 7. Enriching the Data 

### 7.1 Identifying Outliers in terms of quantities of reviews

Usually, we would take Q3 + 1.5 * IQR as an upper boundary and Q1 - 1.5 * IQR as a lower boundary. In this case, however, the lower boundary is negative. Furthermore, calculating Q3 + 1.5 * QR yields a very large value of 688, whereas the maximum amount of reviews per movie is 600:

In [ ]:
review_counts = df["Movie_Title"].value_counts().sort_index()
review_counts = review_counts.to_frame()
review_counts.reset_index(inplace=True)

In [ ]:
review_counts.describe()

In [ ]:
Q1 = review_counts['count'].quantile(0.25)
Q3 = review_counts['count'].quantile(0.75)
IQR = Q3 - Q1
print(f'This is the lower boundary : {Q1-1.5*IQR}\nThis is the upper boundary: {Q3+1.5*IQR}')

Looking at the below boxplot, it makes sense to define the lower_boundary as 0. We also take the upper_boundary as 2 standard deviations above the mean. 

In [ ]:
boxplot = review_counts.boxplot(column=['count'])  

In [ ]:
upper_boundary = review_counts['count'].mean() + 2*review_counts['count'].std()

In [ ]:
review_counts['count'] = review_counts['count'] > upper_boundary

In [ ]:
df = df.merge(review_counts,how='left', on='Movie_Title')
df.rename(columns={'count': 'Outlier_Reviews_Amount_(True/False)'}, inplace=True)

### 7.2 Percentage of difference between the critics' overall score and the audience's overall score 

This could help provide an interesting insight into how much the audience's appreciation of the movie coincides with that of critics'. Some examples of questions : are critics on average more appreciative of Oscars movies than audiences? why? 

In [ ]:
pd.options.mode.chained_assignment = None 
df['Difference_Critics_Audience'] = df['Tomatometer'] - df['Audience_Score']

### 7.3 Movies' box office revenue adjusted for inflation 

These adjusted amounts brought forward to 2024 will allow us to accurately compare them. 

All box office revenue is listed in USD on Rotten Tomatoes. We first download the CPI from the FED's website: https://fred.stlouisfed.org/series/CPIAUCNS and from there adjust the box office amounts to today's value. The file has been provided in the code folder.

In [ ]:
df_inflation = pd.read_csv('CPI_data.csv')
df_inflation.rename(columns={'CUUS0000SA0': 'CPI'}, inplace=True)

In [ ]:
#adjust the dates' format
df_inflation['DATE'].str.split('/')
df_inflation['DATE'] = df_inflation['DATE'].apply(lambda x : x[0:4])
df_inflation.drop_duplicates(subset='DATE', inplace=True)

In [ ]:
df['DATE'] = df['Release_Date_(Theaters)'].str[0:4]

In [ ]:
#CPI_Multiplier defined with reference year being 2023 (we take the same value for 2024 for simplicity's sake)
df_inflation['Multiplier'] = df_inflation['CPI'].iloc[-1] / df_inflation['CPI']
df_inflation.loc[29] = [int(2024), 302.408, 1.000000]

In [ ]:
df = pd.merge(df, df_inflation, how='left', on='DATE')

In [ ]:
df['Box_Office_Adjusted_Inflation_(USD)'] = df['Box_Office_(Gross_USA)']*df['Multiplier']

In [ ]:
#Lastly, we make some minor adjustments before exporting to csv:
df.fillna(0, inplace=True)
df.drop(['Multiplier','CPI','DATE'], axis=1, inplace=True)

df = df[['Movie_Title', 'Release_Date_(Theaters)','Rottentomatoes_URLs', 'Rating', 'Genre', 'Original_Language', 'Director', 
             'Producer','Writer', 'Cast_And_Crew', 'Distributor','Production_Co', 'Synopsis','Runtime', 'Box_Office_(Gross_USA)', 
             'Box_Office_Adjusted_Inflation_(USD)','Audience_Score', 'Tomatometer_State', 'Tomatometer', 'Critics_Concensus',
            'Difference_Critics_Audience','Review_Date','Author', 'Journal','Positive_Negative','Reviews',
             'Outlier_Reviews_Amount_(True/False)']]

In [ ]:
warnings.filterwarnings("ignore")
df['Box_Office_Adjusted_Inflation_(USD)'] = df['Box_Office_Adjusted_Inflation_(USD)'].fillna(0).astype(int)
df['Box_Office_Adjusted_Inflation_(USD)'].replace(0,np.nan, inplace= True)

In [ ]:
df.to_csv('Vedenikova_Vitalia_studentB_stage3.csv', sep=',',index=False, encoding='utf-8-sig')

## Conclusion 

Finally, we obtain a data frame where we have a row for each review. The left-most rows contain all the basic characteristics of the movies (release date, director, producer, etc.), and the right-most rows contain the reviews and related columns. 

We decided to keep the data frame in this format on purpose so that it is easier for Student C to manipulate as needed for the final merge. Indeed, they will not have to think about ways to reset indexes if the information is presented in such a straightforward way. 